In [1]:
import requests

api_url = "https://justjoin.it/api/offers"
r = requests.get(api_url)

In [2]:
import json
import pandas as pd

data = json.loads(r.text)
type(data)

df = pd.DataFrame.from_dict(data)

In [4]:
df = df.drop(["street", "address_text", "marker_icon", "company_url", "latitude", "longitude", "remote_interview", "open_to_hire_ukrainians", "company_logo_url", "multilocation", "way_of_apply"], axis=1)

KeyError: "['street' 'address_text' 'marker_icon' 'company_url' 'latitude'\n 'longitude' 'remote_interview' 'open_to_hire_ukrainians'\n 'company_logo_url' 'multilocation' 'way_of_apply'] not found in axis"

In [13]:
df["employment_types"].map(lambda x: get_types_to_set(x, None))

0        {'type': 'b2b', 'salary': {'from': 10500, 'to'...
1        {'type': 'b2b', 'salary': {'from': 16000, 'to'...
2        {'type': 'permanent', 'salary': {'from': 16000...
3        {'type': 'b2b', 'salary': {'from': 15000, 'to'...
4                    {'type': 'permanent', 'salary': None}
                               ...                        
15641    {'type': 'b2b', 'salary': {'from': 18000, 'to'...
15642    {'type': 'b2b', 'salary': {'from': 18000, 'to'...
15643    {'type': 'b2b', 'salary': {'from': 18000, 'to'...
15644    {'type': 'b2b', 'salary': {'from': 18000, 'to'...
15645    {'type': 'b2b', 'salary': {'from': 24000, 'to'...
Name: employment_types, Length: 15646, dtype: object

In [12]:
def get_types_to_set(dict_list_arg, types_set):
    return dict_list_arg[0]
    for l in dict_list_arg:
        return l[0]